# Stable Diffusion WebUI
*Optimized for RunPod*

This notebook can be run end-to-end to get a working WebUI for Stable Diffusion with support for face correction (GFPGAN) and superresolution upscaling (ESRGAN).

If running on RunPod, assumes you're starting with the RunPod Pytorch container. That said, this notebook does not make use of the built-in preinstalled pytorch as version/dependency incompatibilities between the preinstalled packages and the ones needed by the pipeline make it much easier to start fresh.

This notebook is designed to create a persistent conda environment and model files in the /workspace/ directory, which persists across stopping and starting pods. It could easily be modified to use any similar persistent storage in other services (e.g. a google drive mount in colab).

Ensure you're on a GPU with the appropriate VRAM for what you're trying to generate. GPUs with 16GB or less VRAM will probably only be able to generate a single 512x512 image at a time. Larger GPUs like the A100 are recommended but definitely more expensive.

In [1]:
# Check CUDA stats to make sure we have the correct GPU
!nvidia-smi -L

GPU 0: NVIDIA A100 80GB PCIe (UUID: GPU-d9c7c489-f1a8-03ff-52c7-a405cb020992)


In [2]:
# Update conda
!conda update --force conda -y



         See 'conda update --help' for details about the --force-reinstall
         and --clobber flags.


Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - conda


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _openmp_mutex-5.1          |            1_gnu          21 KB
    ca-certificates-2022.07.19 |       h06a4308_0         124 KB
    cffi-1.15.1                |   py37h74dc2b5_0         227 KB
    conda-4.14.0               |   py37h06a4308_0         909 KB
    cytoolz-0.11.0             |   py37h7b6447c_0         328 KB
    ffmpeg-4.2.2               |       h20bf706_0        59.6 MB
    ld_impl_linux-64-2.38      |       h1181459_1         654 KB
    lerc-3.0                   |       h295c915_0         196 KB
    libdeflate-1.8             |       h7f8727e_5          51 KB
    libgcc-ng-11.2.0           |       h

In [3]:
# clean up cached conda packages from whatever came with the docker container
!conda clean --all -y

Will remove 32 (89.5 MB) tarball(s).
Will remove 1 index cache(s).
Will remove 1 (103 KB) package(s).
There are no tempfile(s) to remove.
There are no logfile(s) to remove.


In [5]:
# Set your working directory here (usually whatever persistent storage mount is available to your runtime.
#For RunPod it will be /workspace/
wd = "/workspace"
%cd {wd}

/workspace


In [6]:
# create new conda env
!mkdir -p {wd}/sd_env
!conda create -p {wd}/sd_env/ python=3.8 -y

Solving environment: done

## Package Plan ##

  environment location: /workspace/sd_env

  added / updated specs:
    - python=3.8


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _libgcc_mutex-0.1          |             main           3 KB
    _openmp_mutex-5.1          |            1_gnu          21 KB
    certifi-2022.6.15          |   py38h06a4308_0         153 KB
    pip-22.1.2                 |   py38h06a4308_0         2.5 MB
    python-3.8.13              |       h12debd9_0        18.8 MB
    setuptools-63.4.1          |   py38h06a4308_0         1.1 MB
    ------------------------------------------------------------
                                           Total:        22.5 MB

The following NEW packages will be INSTALLED:

  _libgcc_mutex      pkgs/main/linux-64::_libgcc_mutex-0.1-main
  _openmp_mutex      pkgs/main/linux-64::_openmp_mutex-5.1-1_gnu
  ca-certificates    pkgs

In [7]:
conda_env = "/workspace/sd_env"
pip_path = conda_env + "/bin/pip"
python_path = conda_env + "/bin/python"


In [8]:
!git clone https://github.com/hlky/stable-diffusion
%cd stable-diffusion
!{pip_path} install -e .

Cloning into 'stable-diffusion'...
remote: Enumerating objects: 995, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 995 (delta 30), reused 72 (delta 29), pack-reused 917
Receiving objects: 100% (995/995), 42.67 MiB | 26.27 MiB/s, done.
Resolving deltas: 100% (480/480), done.
/workspace/stable-diffusion
Obtaining file:///workspace/stable-diffusion
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 7.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 86.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 54.3 MB/s eta 0:00:00
  Running setup.py develop for latent-diffusion


In [9]:
# make sure we didnt break cuda
!nvidia-smi -L

GPU 0: NVIDIA A100 80GB PCIe (UUID: GPU-d9c7c489-f1a8-03ff-52c7-a405cb020992)


In [10]:
# Force the correct pytorch version and dependencies before proceeding. 
# Incorrect versions of pytorch/torchvision/torchtext will put you into dependency hell
!conda install -f -p {conda_env} -c pytorch pytorch=1.11.0 -y



         See 'conda install --help' for details about the --force-reinstall
         and --clobber flags.


Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /workspace/sd_env

  added / updated specs:
    - pytorch=1.11.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    blas-1.0                   |              mkl           6 KB
    cudatoolkit-11.3.1         |       h2bc3f7f_2       549.3 MB
    intel-openmp-2022.0.1      |    h06a4308_3633         4.2 MB
    libuv-1.40.0               |       h7b6447c_0         736 KB
    mkl-2022.0.1               |     h06a4308_117       127.7 MB
    pytorch-1.11.0             |py3.8_cuda11.3_cudnn8.2.0_0        1.02 GB  pytorch
    pytorch-mutex-1.0          |             cuda           3 KB  pytorch
    typing_extensions-4.3.0    |   py38h

In [11]:
# Install all the dependencies for stable diffusion webui into our conda env
!conda env update -p {conda_env} -f /workspace/stable-diffusion/environment.yaml 

Solving environment: done

pip-20.3.3           | 1.8 MB    | ##################################### | 100% 
urllib3-1.26.11      | 182 KB    | ##################################### | 100% 
krb5-1.19.2          | 1.2 MB    | ##################################### | 100% 
requests-2.28.1      | 92 KB     | ##################################### | 100% 
cffi-1.15.1          | 228 KB    | ##################################### | 100% 
cryptography-37.0.1  | 1.3 MB    | ##################################### | 100% 
python-3.8.5         | 49.3 MB   | ##################################### | 100% 
torchvision-0.12.0   | 27.7 MB   | ##################################### | 100% 
mkl-2020.2           | 138.3 MB  | ##################################### | 100% 
curl-7.84.0          | 79 KB     | ##################################### | 100% 
mkl_fft-1.3.0        | 177 KB    | ##################################### | 100% 
pillow-9.2.0         | 666 KB    | ##################################### | 100% 
l

### Build upscalers support
**GFPGAN** Automatically correct distorted faces with a built-in GFPGAN option, fixes them in less than half a second  
**ESRGAN** Boosts the resolution of images with a built-in RealESRGAN option

In [13]:

add_CFP = True
add_ESR = True

if add_CFP:
    %cd {wd}/stable-diffusion/src/gfpgan/
    !{pip_path} install basicsr facexlib yapf lmdb opencv-python pyyaml tb-nightly --no-deps
    !{python_path} setup.py develop
    !wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth -P experiments/pretrained_models
if add_ESR:
    !{pip_path} install realesrgan
    %cd {wd}/stable-diffusion/src/realesrgan/
    !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P experiments/pretrained_models
    !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.2.4/RealESRGAN_x4plus_anime_6B.pth -P experiments/pretrained_models

%cd {wd}/stable-diffusion/
!wget https://github.com/matomo-org/travis-scripts/blob/master/fonts/Arial.ttf?raw=true -O arial.ttf
%cd {wd}

/workspace/stable-diffusion/src/gfpgan
/workspace/sd_env/lib/python3.8/site-packages/setuptools/installer.py:27: SetuptoolsDeprecationWarning: setuptools.installer is deprecated. Requirements should be satisfied by a PEP 517 installer.
  warnings.warn(
running develop
/workspace/sd_env/lib/python3.8/site-packages/setuptools/command/easy_install.py:144: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/workspace/sd_env/lib/python3.8/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
running egg_info
writing gfpgan.egg-info/PKG-INFO
writing dependency_links to gfpgan.egg-info/dependency_links.txt
writing requirements to gfpgan.egg-info/requires.txt
writing top-level names to gfpgan.egg-info/top_level.txt
reading manifest file 'gfpgan.egg-info/SOURCES.txt'
reading manifest templa

In [16]:
# we need glib to run esrgan/gfpgan
!conda install -p {conda_env} glib -y

Solving environment: done

## Package Plan ##

  environment location: /workspace/sd_env

  added / updated specs:
    - glib


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    glib-2.69.1                |       h4ff587b_1         1.7 MB
    pcre-8.45                  |       h295c915_0         207 KB
    ------------------------------------------------------------
                                           Total:         1.9 MB

The following NEW packages will be INSTALLED:

  glib               pkgs/main/linux-64::glib-2.69.1-h4ff587b_1
  pcre               pkgs/main/linux-64::pcre-8.45-h295c915_0



glib-2.69.1          | 1.7 MB    | ##################################### | 100% 
pcre-8.45            | 207 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Retrieving notices: ...working... done


# Fetch Model Weights

In [17]:
# Feel free to change destination paths here
models_path = wd +"/models"
output_path = wd +"/output"

import os
os.makedirs(models_path, exist_ok=True)
os.makedirs(output_path, exist_ok=True)

In [18]:
# Snag the weights from a publicly hosted copy. If this goes down, use huggingface.
!curl https://www.googleapis.com/storage/v1/b/aai-blog-files/o/sd-v1-4.ckpt?alt=media > {wd}/models/sd-v1-4.ckpt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4067M  100 4067M    0     0  67.6M      0  0:01:00  0:01:00 --:--:-- 64.9M


## Time to run the Stable Diffusion webui

### Set the variables below to adjust configuration

In [19]:
#@markdown # Launch preferences - Advanced
share_password="" #@param {type:"string"}
#@markdown * Add a password to your webui
defaults="configs/webui/webui.yaml" #@param {type:"string"}
#@markdown * path to configuration file providing UI defaults, uses same format as cli parameter)  
#@markdown Edit this file if you want to change the default settings UI launches with

#@markdown ---
save_metadata = False #@param {type:"boolean"}
#@markdown * Whether to embed the generation parameters in the sample images
skip_grid = False #@param {type:"boolean"}
#@markdown * Do not save a grid, only individual samples. Helpful when evaluating lots of samples
skip_save = False #@param {type:"boolean"}
#@markdown * Do not save individual samples as files. For speed measurements
optimized = False #@param {type:"boolean"}
#@markdown * Load the model onto the device piecemeal instead of all at once to reduce VRAM usage at the cost of performance
optimized_turbo = True #@param {type:"boolean"}
#@markdown * Alternative optimization mode that does not save as much VRAM but runs siginificantly faster
no_verify_input = False #@param {type:"boolean"}
#@markdown * Do not verify input to check if it's too long
no_half = False #@param {type:"boolean"}
#@markdown * Do not switch the model to 16-bit floats
no_progressbar_hiding = True #@param {type:"boolean"}
#@markdown * Do not hide progressbar in gradio UI
extra_models_cpu = False #@param {type:"boolean"}
#@markdown * Run extra models (GFGPAN/ESRGAN) on cpu
esrgan_cpu = True #@param {type:"boolean"}
#@markdown * run ESRGAN on cpu
gfpgan_cpu = True #@param {type:"boolean"}
#@markdown * run GFPGAN on cpu


run_string_with_variables = {
 '--save-metadata': f'{save_metadata}',
 '--skip-grid': f'{skip_grid}',
 '--skip-save': f'{skip_save}',
 '--optimized': f'{optimized}',
 '--optimized-turbo': f'{optimized_turbo}',
 '--no-verify-input': f'{no_verify_input}',
 '--no-half': f'{no_half}',
 '--no-progressbar-hiding': f'{no_progressbar_hiding}',
 '--extra-models-cpu': f'{extra_models_cpu}',
 '--esrgan-cpu': f'{esrgan_cpu}',
 '--gfpgan-cpu': f'{gfpgan_cpu}'}

only_true_vars = {k for (k,v) in run_string_with_variables.items() if v == 'True'}
vars = " ".join(only_true_vars)

## Launch The WebUI!

In [21]:

#@markdown # Important - click the public URL to launch WebUI in another tab
#@markdown ![](https://user-images.githubusercontent.com/463317/187105407-dd9b0f4e-c8da-49d3-8c78-1767f5c9aa83.jpg)

%cd {wd}/stable-diffusion
!{python_path} {wd}/stable-diffusion/scripts/webui.py \
--ckpt '{models_path}/sd-v1-4.ckpt' \
--outdir '{output_path}' \
--share {vars}

/workspace/stable-diffusion
Downloading: "https://github.com/xinntao/facexlib/releases/download/v0.1.0/detection_Resnet50_Final.pth" to /workspace/stable-diffusion/src/facexlib/facexlib/weights/detection_Resnet50_Final.pth

100%|████████████████████████████████████████| 104M/104M [00:03<00:00, 27.5MB/s]
Downloading: "https://github.com/xinntao/facexlib/releases/download/v0.2.2/parsing_parsenet.pth" to /workspace/stable-diffusion/src/facexlib/facexlib/weights/parsing_parsenet.pth

100%|███████████████████████████████████████| 81.4M/81.4M [00:00<00:00, 111MB/s]
Loaded GFPGAN
Loaded RealESRGAN with model RealESRGAN_x4plus
Loading model from /workspace/models/sd-v1-4.ckpt
Global Step: 470000
UNet: Running in eps-prediction mode
^C
Traceback (most recent call last):
  File "/workspace/stable-diffusion/scripts/webui.py", line 359, in <module>
    _, _ = model.load_state_dict(sd, strict=False)
  File "/workspace/sd_env/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1483, in loa

^ I interrupted the code above before it finished, but when complete it will give you a public Gradio URL you can use to access your webapp. Have fun!